# "Uploading" PDFs to Claude Via the API

One really nice feature of [Claude.ai](https://www.claude.ai) is the ability to upload PDFs. Let's mock up that feature in a notebook, and then test it out by summarizing a long PDF.

In [ ]:
!pip install pypdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2039k  100 2039k    0     0  3576k      0 --:--:-- --:--:-- --:--:-- 3577k


Now, we'll use pypdf to read the pdf. It's not identical to what Claude.ai uses behind the scenes, but it's pretty close.

In [6]:
from pypdf import PdfReader

reader = PdfReader("Test_Python_1.pdf")
number_of_pages = len(reader.pages)
text = ''.join([page.extract_text() for page in reader.pages])
print(text[:2155])

FileNotFoundError: [Errno 2] No such file or directory: 'Test_Python_1.pdf'

With the paper downloaded and in memory, we can ask Claude to perform various fun tasks with it.

In [2]:
import os
import sys
import anthropic
import dotenv
import nest_asyncio
nest_asyncio.apply()  # This is a workaround for asyncio + jupyter notebooks
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))


dotenv.load_dotenv()

API_KEY = os.environ['ANTHROPIC_API_KEY']
CLIENT = anthropic.Client(api_key=API_KEY)
def get_completion(client, prompt, max_tokens=3000, model='claude-2'):
    return client.completions.create(
        prompt=prompt, max_tokens_to_sample=max_tokens, model=model
    ).completion

In [5]:
choice = "Serious and Professional", "Casual and Fun"
completion = get_completion(CLIENT, 
    f"""\n\nHuman: You are a tutor named Jack&Jones that specializes in Personalized Teaching Asssitant as the given {{subject}}. You are patient, helpful and make learning fun with occasional jokes. You have a knack for creating multiple choice quizzes, precise guideline and can provide tips on any aspect of the topic with rigor.
All of your responses and everything you know about is captured in the {text} document. The {text} is an extract from python code syntax. 

Now generate me a quiz based on the given {text}. Choose the below output format for the quiz:

1. With <quiz> Your Visual Python Learner </>quiz intro you should ask the user if they want to begin.
2. A quiz consists of 5 questions all based on at least one highlight.
3. Question format would be according to user choice : {choice}.
4. The quiz should have an intro that explains that there are 5 questions about Cultural Differences 
5. Each question should have text for the question, four answer options (only one of which is the correct answer) each should have a letter from a to d next to the text for that answer option. The letter should have a dash on the right side of it to separate it from the text answer. Each question should have a response that confirms whether the answer was correct or not and provide the answer coupled with the letter that was used with it.
 
Assistant: Can I think step-by-step and like a rubber duck?

Human: Yes, please do.

+Assistant:


Human: Make an explanation in layman terms of the given answer.

+Assistant:


"""
)
print(completion)

NameError: name 'text' is not defined